# Notebook to generate n=100000 test jobs

In [ ]:
import pyvo
from astropy.table import Table
from lsst.rsp import get_tap_service
from concurrent.futures import ThreadPoolExecutor
import itertools

In [ ]:
N = 100000

In [ ]:
rsp_tap = get_tap_service("tap")
assert rsp_tap is not None

### 1.2. Create upload data (as a string)

In [ ]:
query = """
        SELECT INVALID
        """

In [ ]:
def run_single_job():
    job = rsp_tap.submit_job(query)
    job.run()
    job.wait(phases=['COMPLETED', 'ERROR'])
    return job

batch_size = 5
with ThreadPoolExecutor(max_workers=batch_size) as executor:
    for i in range(0, N, batch_size):
        current_batch_size = min(batch_size, N - i)
        futures = [executor.submit(run_single_job) for _ in range(current_batch_size)]
        
        for future in futures:
            future.result()
        
        print(f"Completed batch {i//batch_size + 1}")